In [26]:
!wget http://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip

URL transformed to HTTPS due to an HSTS policy
--2021-01-25 13:14:27--  https://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip
Resolving cs.stanford.edu (cs.stanford.edu)... 171.64.64.64
Connecting to cs.stanford.edu (cs.stanford.edu)|171.64.64.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 81363704 (78M) [application/zip]
Saving to: ‘trainingandtestdata.zip.1’

trainingandtestdata 100%[===================>]  77.59M  36.0MB/s    in 2.2s    

2021-01-25 13:14:31 (36.0 MB/s) - ‘trainingandtestdata.zip.1’ saved [81363704/81363704]



In [3]:
!unzip -q trainingandtestdata.zip -d tweets

In [7]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /home/azureuser/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [9]:
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer(strip_handles=True, reduce_len=True)
from nltk.stem import WordNetLemmatizer 
from nltk.stem import PorterStemmer 
import string
import re
import numpy as np
import pandas as pd
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from multiprocessing import cpu_count
stop_words = set(stopwords.words('english')) 
dataset= pd.read_csv('tweets/training.1600000.processed.noemoticon.csv', names=["Col1","Col2","Col3","Col4","Col5","Col6"], encoding='ISO-8859-1') 


############ LEMMITIZER ###############
#lemmatizer = WordNetLemmatizer()
#dataset['Col6']=dataset['Col6'].apply(lambda x: " ".join(lemmatizer.lemmatize(i) for i in tknzr.tokenize(x) if not i in stop_words ))
#######################################

########### STEMMER ##################
ps = PorterStemmer()
dataset['Col6']=dataset['Col6'].apply(lambda x: " ".join(ps.stem(i) for i in tknzr.tokenize(x) if not i in stop_words ))
######################################

dataset['Col6']=dataset['Col6'].apply(lambda x: re.sub("@S+|www?:\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+",' ',x))
dataset['Col1']= dataset['Col1'].replace(4,1)
dataset['Col6']=dataset['Col6'].str.replace('\d+', '').apply(lambda x:str(x).translate(str.maketrans('', '', string.punctuation)).lower().rstrip())
dataset= dataset[["Col1","Col6"]]
dataset= dataset.dropna(axis=0,how="any")
ds= dataset

In [10]:
from multiprocessing import cpu_count
from gensim.models import FastText, Word2Vec

tokenized= ds
word_tokens= tokenized['Col6'].apply(lambda x: [w for w in word_tokenize(x)])
nrDimentsions=100

######## Word2Text ####################
#model = Word2Vec(word_tokens, min_count=1, size=nrDimentsions,workers=cpu_count())
#######################################

######## FastText ######################
model = FastText(word_tokens, min_count=1, size=nrDimentsions,workers=cpu_count())
########################################
model.train(word_tokens,total_examples=len(word_tokens),epochs=7, verbose=1)

words = list(model.wv.vocab)
print(words)

# save the vectors in a new matrix
embedding_matrix = np.zeros((len(model.wv.vocab) + 1, nrDimentsions))
for i, vec in enumerate(model.wv.vectors):
    embedding_matrix[i] = vec

['awww', 'that', 'bummer', 'you', 'shoulda', 'got', 'david', 'carr', 'third', 'day', 'd', 'upset', 'can', 't', 'updat', 'facebook', 'text', 'might', 'cri', 'result', 'school', 'today', 'also', 'blah', 'i', 'dive', 'mani', 'time', 'ball', 'manag', 'save', 'the', 'rest', 'go', 'bound', 'whole', 'bodi', 'feel', 'itchi', 'like', 'fire', 'behav', 'm', 'mad', 'see', 'crew', 'need', 'hug', 'hey', 'long', 'ye', 'rain', 'bit', 'lol', 'fine', 'thank', 'how', 'nope', 'que', 'muera', 'spring', 'break', 'plain', 'citi', 'snow', 're', 'pierc', 'ear', 'bear', 'watch', 'and', 'thought', 'ua', 'loss', 'embarrass', 'it', 'count', 'idk', 'either', 'never', 'talk', 'anymor', 'would', 'v', 'first', 'gun', 'realli', 'though', 'zac', 'snyder', 'doucheclown', 'wish', 'miss', 'premier', 'holli', 'death', 'scene', 'hurt', 'sever', 'film', 'wri', 'director', 'cut', 'file', 'tax', 'ahh', 'ive', 'alway', 'want', 'rent', 'love', 'soundtrack', 'oh', 'dear', 'were', 'drink', 'forgotten', 'tabl', 'get', 'much', 'done'

In [12]:
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# how many features should the tokenizer extract
features = embedding_matrix.shape[0]
tokenizer = Tokenizer(num_words = features)
# fit the tokenizer on our text
tokenizer.fit_on_texts(ds['Col6'])

# get all words that the tokenizer knows
word_index = tokenizer.word_index

# put the tokens in a matrix
X = tokenizer.texts_to_sequences(ds['Col6'])
X = pad_sequences(X)

# prepare the labels
y = pd.get_dummies(ds['Col1'])

# split in train and test
X_train, X_test, y_train, y_test = train_test_split(X, ds['Col1'], test_size=0.1, random_state=42)

Using TensorFlow backend.


In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Dropout, Activation,Conv1D,MaxPooling1D


mod = Sequential()
mod.add(Embedding(len(model.wv.vocab)+1 ,nrDimentsions,weights=[embedding_matrix],input_length=X.shape[1],trainable=False))
mod.add(Dropout(0.25))
mod.add(LSTM(500, activation="relu"))

mod.add(Dense(1))
mod.add(Activation('sigmoid'))
mod.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [29]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
mod.fit(X_train,y_train,epochs= 20, batch_size=32, verbose=1, validation_split=0.2, callbacks= [EarlyStopping(
    monitor='val_accuracy', min_delta=1e-4,restore_best_weights=True, patience=3)])

Epoch 1/20
36000/36000 [==============================] - 1016s 28ms/step - loss: 0.4941 - accuracy: 0.7565 - val_loss: 0.4775 - val_accuracy: 0.7687
Epoch 2/20
36000/36000 [==============================] - 1015s 28ms/step - loss: 0.4916 - accuracy: 0.7583 - val_loss: 0.4762 - val_accuracy: 0.7693
Epoch 3/20
36000/36000 [==============================] - 1006s 28ms/step - loss: 0.4894 - accuracy: 0.7595 - val_loss: 0.4742 - val_accuracy: 0.7707
Epoch 4/20
36000/36000 [==============================] - 984s 27ms/step - loss: 0.4887 - accuracy: 0.7600 - val_loss: 0.4740 - val_accuracy: 0.7712
Epoch 5/20
36000/36000 [==============================] - 986s 27ms/step - loss: 0.4874 - accuracy: 0.7608 - val_loss: 0.4742 - val_accuracy: 0.7708
Epoch 6/20
36000/36000 [==============================] - 984s 27ms/step - loss: 0.4871 - accuracy: 0.7614 - val_loss: 0.4723 - val_accuracy: 0.7723
Epoch 7/20
36000/36000 [==============================] - 989s 27ms/step - loss: 0.4854 - accuracy: 0.7

In [32]:
acc= mod.evaluate(X_test,y_test)
print(acc)


5000/5000 [==============================] - 59s 12ms/step - loss: 0.4704 - accuracy: 0.7727
[0.47038355469703674, 0.7727375030517578]


In [31]:
from sklearn.metrics import roc_auc_score
y_scores = mod.predict_classes(X_test)
roc_auc_score(y_test, y_scores)

0.772732896167419